# 인벤 게시판 댓글 크롤링

In [ ]:
"""
인벤 오픈 이슈 갤러리 게시글의 댓글을 수집한다.
"""

import requests
from bs4 import BeautifulSoup
import time
import ujson


LIST_PAGE_URL_TMPL = "http://www.inven.co.kr/board/powerbbs.php?come_idx=2097&query=list&my=&category=&category2=&sort=PID&orderby=&name=&subject=&content=&keyword=&sterm=&eq=&iskin=webzine&mskin=&p={}"
COMMENT_REQ_URL = "http://www.inven.co.kr/common/board/comment.xml.php?dummy=1502246098813"
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) " + \
             "AppleWebKit/537.36 (KHTML, like Gecko) " + \
             "Chrome/37.0.2062.94 Safari/537.36"
HEADERS = {"User-Agent": USER_AGENT}
PAGING_STOP_PAT = "현재 등록된 게시물이 없습니다."
SLEEP_TIME = 2


def crawl_comments(output_file_name):
    """인벤 오픈 이슈 갤러리 게시글의 댓글을 수집하여 주어진 이름의 파일에 저장한다."""
    
    with open(output_file_name, "w", encoding="utf-8") as output_file:   
        page_num = 1
        while True:
            post_ids = fetch_post_ids(page_num)
            
            if post_ids is None:
                break
                
            fetch_store_comments(output_file, post_ids)
            page_num += 1
            break
        
        
def fetch_post_ids(page_num):
    """주어진 페이지 번호의 목록 페이지에서 댓글이 있는 게시물 ID들을 수집하여 돌려준다."""

    list_page_url = gen_list_page_url(page_num)
    html = get_html(list_page_url)

    if paging_done(html):
        return None

    soup = get_soup(html)
    post_ids = ext_post_ids(soup)
    
    return post_ids


def gen_list_page_url(page_num):
    """게시글 목록 페이지 URL을 생성하여 돌려준다."""
    
    list_page_url = LIST_PAGE_URL_TMPL.format(page_num)
    
    return list_page_url


def get_html(url):
    """주어진 URL에 접근하여 HTML 텍스트를 읽어서 돌려준다."""

    response = requests.get(url, headers=HEADERS)
    html = response.text
    pause()

    return html


def pause():
    """정해진 시간만큼 쉰다."""

    time.sleep(SLEEP_TIME)
    
    
def get_soup(html):
    """주어진 HTML 텍스트를 soup 객체로 만들어 돌려준다."""
    
    soup = BeautifulSoup(html, "lxml")
    
    return soup


def paging_done(html):
    """페이징이 완료되었는지를 판단한다."""

    if PAGING_STOP_PAT in html:
        return True

    return False


def ext_post_ids(soup):
    """주어진 soup 객체에서 댓글이 있는 게시물 ID들을 추출하여 돌려준다."""
    
    post_id_elems = soup.find_all("span", class_="sj_cm")
    post_ids = [elem["data-cmt-uid"] for elem in post_id_elems]

    return post_ids


def fetch_store_comments(output_file, post_ids):
    """주어진 게시글 ID의 댓글들을 수집하여 출력 파일에 저장한다."""
    
    for post_id in post_ids:
        xml = get_comment_xml(post_id)
        soup = get_soup(xml)
        authors = ext_authors(soup)
        date_times = ext_date_times(soup)
        bodies = ext_bodies(soup)
        write_comments(output_file, post_id, authors, date_times, bodies)   
        
        
def get_comment_xml(post_id):
    """주어진 게시글 ID에 대한 댓글 XML 텍스트를 요청하여 돌려준다."""
    
    req_data = gen_comment_req_data(post_id)
    resp = requests.post(COMMENTS_REQ_URL, data=req_data)
    xml = resp.text
    pause()
    
    return xml


def gen_comment_req_data(post_id):
    """주어진 게시글 ID의 댓글 페이지 요청 데이터를 생성하여 돌려준다."""
    
    req_data = {
        "comeidx": 2097,
        "articlecode": post_id,
        "sortorder": "date",
        "uploadurl": "",
        "imageposition": "",
        "act": "list",
        "out": "xml",
        "replynick": "",
        "replyidx": 0   
    }     

    return req_data
        
    
def ext_authors(soup):
    """주어진 soup 객체에서 게시자들을 추출하여 돌려준다."""
    
    author_elems = soup.find_all("o_name")
    authors = [elem.string for elem in author_elems]
    
    return authors

            
def ext_date_times(soup):
    """주어진 soup 객체에서 게시 날짜, 시간을 추출하여 돌려준다."""
   
    date_time_elems = soup.find_all("o_date")
    date_times = [elem.string for elem in date_time_elems]
    
    return date_times
    
    
def ext_bodies(soup):
    """주어진 soup 객체에서 댓글 본문을 추출하여 돌려준다."""
    
    body_elems = soup.find_all("o_comment")
    bodies = [elem.string.replace("&nbsp;", " ") for elem in body_elems]

    return bodies
       

def write_comments(output_file, post_id, authors, date_times, bodies):
    """댓글을 JSON 문자열로 만들어 출력 파일에 기록한다."""
                   
    for author, date_time, body in zip(authors, date_times, bodies):
        comment = {"post_id": post_id, "author": author, "date_time": date_time, 
                   "body": body}
        json_str = ujson.dumps(comment, ensure_ascii=False)
        print(json_str, file=output_file)
              
    output_file.flush()


def main():
    """인벤 오픈 이슈 갤러리 게시글의 댓글을 수집하여 저장한다."""
    
    output_file_name = "../data/crawling/inven-open-issue-comments.txt"
    crawl_comments(output_file_name)
    
#
# 실행
#
    
main()      